In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1

mps


pip install 'aif360[AdversarialDebiasing]'


In [2]:
# read in data and split
X = torch.load('inputs/rfw_resnet50_face_embeddings.pt').cpu()
y = torch.load('inputs/rfw_resnet50_labels.pt').cpu()
df = pd.read_csv('inputs/rfw_resnet50_df.csv')
df['reference_ethnicity'] = df['reference_ethnicity'].str.lower()
ethnicity = df['reference_ethnicity']

df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
0,m.0c7mh2,m.0c7mh2,african,African,1.0
1,m.0c7mh2,m.0c7mh2,african,African,1.0
2,m.026tq86,m.026tq86,african,African,1.0
3,m.026tq86,m.026tq86,african,African,1.0
4,m.02wz3nc,m.02wz3nc,african,African,1.0
...,...,...,...,...,...
29311,m.0402tg,m.01npnk3,caucasian,Caucasian,0.0
29312,m.05pbbnj,m.02rrb2n,caucasian,Caucasian,0.0
29313,m.09j6df,m.07kcsqd,african,African,0.0
29314,m.0fhrbz,m.025zgjt,african,African,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]


train_ethnicity = ethnicity[train_split].values
train_ethnicity[train_ethnicity=='caucasian'] = 0
train_ethnicity[train_ethnicity=='african'] = 1
train_ethnicity = train_ethnicity.astype(int)

ethnicity_val = ethnicity[val_split].values
ethnicity_val[ethnicity_val=='caucasian'] = 0
ethnicity_val[ethnicity_val=='african'] = 1
ethnicity_val = ethnicity_val.astype(int)

test_ethnicity = ethnicity[test_split].values
test_ethnicity[test_ethnicity=='caucasian'] = 0
test_ethnicity[test_ethnicity=='african'] = 1
test_ethnicity = test_ethnicity.astype(int)

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(train_matches['reference_ethnicity'].value_counts())

african_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='african']
caucasian_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='caucasian']

african_matches = train_matches[train_matches['reference_ethnicity']=='african']
caucasian_matches = train_matches[train_matches['reference_ethnicity']=='caucasian']

non matches
african      4492
caucasian    4376
Name: reference_ethnicity, dtype: int64
matches
african      4396
caucasian    4325
Name: reference_ethnicity, dtype: int64


In [5]:
np.random.seed(random_state)
african_matches_sub_idx = african_matches.index[np.random.choice(len(african_matches.index), size=4325, replace=False)]
caucasian_non_matches_sub_idx = caucasian_non_matches.index[np.random.choice(len(caucasian_non_matches.index), size=4325, replace=False)]
african_non_matches_sub_idx = african_non_matches.index[np.random.choice(len(african_non_matches.index), size=4325, replace=False)]
caucasian_matches_sub_idx = caucasian_matches.index

X_train = torch.cat([X[african_matches_sub_idx],X[caucasian_matches_sub_idx],X[african_non_matches_sub_idx],X[caucasian_non_matches_sub_idx]])
y_train = torch.cat([y[african_matches_sub_idx],y[caucasian_matches_sub_idx],y[african_non_matches_sub_idx],y[caucasian_non_matches_sub_idx]])
ethnicity_train = np.concatenate([ethnicity[african_matches_sub_idx],ethnicity[caucasian_matches_sub_idx],ethnicity[african_non_matches_sub_idx],ethnicity[caucasian_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[african_matches_sub_idx],df.iloc[caucasian_matches_sub_idx],df.iloc[african_non_matches_sub_idx],df.iloc[caucasian_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
6419,m.03j6s0,m.03j6s0,african,African,1.0
5541,m.01qd5bm,m.01qd5bm,african,African,1.0
5881,m.02qdp4v,m.02qdp4v,african,African,1.0
229,m.0272001,m.0272001,african,African,1.0
5264,m.05f962,m.05f962,african,African,1.0
...,...,...,...,...,...
18810,m.0g3f1p,m.01kxfvq,caucasian,Caucasian,0.0
20550,m.07gb0p,m.0bgwxt,caucasian,Caucasian,0.0
24018,m.07pyvy,m.01y7m1,caucasian,Caucasian,0.0
20879,m.02qg85,m.070p26,caucasian,Caucasian,0.0


In [6]:
test_split, holdout_split = train_test_split(np.arange(5864),test_size=0.1, random_state=random_state)
X_test = test_X[test_split]
X_holdout = test_X[holdout_split]
y_test = test_y[test_split]
y_holdout = test_y[test_split]
ethnicity_test = test_ethnicity[holdout_split]
ethnicity_holdout = test_ethnicity[holdout_split]
df_test = test_df.iloc[test_split]
df_holdout = test_df.iloc[holdout_split]
df_holdout

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
26599,m.08dyjb,m.01dysq,african,African,0.0
13208,m.05nqz8,m.05nqz8,caucasian,Caucasian,1.0
7459,m.0chlpf,m.0chlpf,caucasian,Caucasian,1.0
4747,m.0gzkxh,m.0gzkxh,african,African,1.0
22098,m.01tl0vh,m.027g5hs,african,African,0.0
...,...,...,...,...,...
27224,m.0glqs_8,m.05972y,african,African,0.0
5728,m.06w9s22,m.06w9s22,african,African,1.0
18206,m.04zvsy6,m.06tvw7,caucasian,Caucasian,0.0
25909,m.0h_dpyh,m.04d_dnm,african,African,0.0


In [7]:
df_test

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
4330,m.027ptzd,m.027ptzd,african,African,1.0
19781,m.01kq4tc,m.06l8hh,african,African,0.0
14300,m.03yly5c,m.03yly5c,caucasian,Caucasian,1.0
25119,m.0b6jzk1,m.09g4qt,african,African,0.0
8829,m.0c2213,m.0c2213,caucasian,Caucasian,1.0
...,...,...,...,...,...
16654,m.0855sw,m.01wgjkd,african,African,0.0
19411,m.04gqvj2,m.01vs73g,african,African,0.0
13763,m.047md3x,m.047md3x,caucasian,Caucasian,1.0
12033,m.05pg_g,m.05pg_g,caucasian,Caucasian,1.0


In [8]:
val_matches = val_df[val_df.labels==1]
val_non_matches = val_df[val_df.labels==0]
print('non matches')
print(val_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(val_matches['reference_ethnicity'].value_counts())

val_african_non_matches = val_non_matches[val_non_matches['reference_ethnicity']=='african']
val_african_non_matches = val_non_matches[val_non_matches['reference_ethnicity']=='african']

val_african_matches = val_matches[val_matches['reference_ethnicity']=='african']
val_african_matches = val_matches[val_matches['reference_ethnicity']=='african']

non matches
caucasian    1460
african      1454
Name: reference_ethnicity, dtype: int64
matches
caucasian    1480
african      1469
Name: reference_ethnicity, dtype: int64


In [9]:
test_matches = df_test[df_test.labels==1]
test_non_matches = df_test[df_test.labels==0]
print('non matches')
print(test_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(test_matches['reference_ethnicity'].value_counts())

test_african_non_matches = test_non_matches[test_non_matches['reference_ethnicity']=='african']
test_african_non_matches = test_non_matches[test_non_matches['reference_ethnicity']=='african']

test_african_matches = test_matches[test_matches['reference_ethnicity']=='african']
test_african_matches = test_matches[test_matches['reference_ethnicity']=='african']

non matches
african      1376
caucasian    1230
Name: reference_ethnicity, dtype: int64
matches
african      1389
caucasian    1282
Name: reference_ethnicity, dtype: int64


In [10]:
holdout_matches = df_holdout[df_holdout.labels==1]
holdout_non_matches = df_holdout[df_holdout.labels==0]
print('non matches')
print(holdout_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(holdout_matches['reference_ethnicity'].value_counts())

holdout_african_non_matches = holdout_non_matches[holdout_non_matches['reference_ethnicity']=='african']
holdout_african_non_matches = holdout_non_matches[holdout_non_matches['reference_ethnicity']=='african']

holdout_african_matches = holdout_matches[holdout_matches['reference_ethnicity']=='african']
holdout_african_matches = holdout_matches[holdout_matches['reference_ethnicity']=='african']

non matches
african      142
caucasian    128
Name: reference_ethnicity, dtype: int64
matches
african      166
caucasian    151
Name: reference_ethnicity, dtype: int64


In [11]:
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2

In [ ]:
eer_thresh = 0.495

In [11]:
val_cos = cos_sim(X_val[:,:2048],X_val[:,2048:])
eer_thresh = 0.495

eer_pred = (val_cos>eer_thresh)*1
acc_pred = (val_cos>acc_thresh)*1
df_bias = df_val.drop(['reference_identity','candidate_identity','reference_ethnicity','candidate_ethnicity','candidate_gender'],axis=1)
df_bias['eer_pred'] = eer_pred
df_bias['acc_pred'] = acc_pred
df_bias_female = df_bias[df_bias['reference_gender']=='female']
df_bias_male = df_bias[df_bias['reference_gender']=='male']

female_eer_tn, female_eer_fp, female_eer_fn, female_eer_tp, female_eer_acc = confusion_mat(df_bias_female['eer_pred'], df_bias_female['labels'])
male_eer_tn, male_eer_fp, male_eer_fn, male_eer_tp, male_eer_acc = confusion_mat(df_bias_male['eer_pred'], df_bias_male['labels'])

female_acc_tn, female_acc_fp, female_acc_fn, female_acc_tp, female_acc_acc = confusion_mat(df_bias_female['acc_pred'], df_bias_female['labels'])
male_acc_tn, male_acc_fp, male_acc_fn, male_acc_tp, male_acc_acc = confusion_mat(df_bias_male['acc_pred'], df_bias_male['labels'])

female_eer_tnr = female_eer_tn/(female_eer_tn+female_eer_fp)
female_eer_tpr = female_eer_tp/(female_eer_tp+female_eer_fn)
female_eer_fnr = female_eer_fn/(female_eer_fn+female_eer_tp)
female_eer_fpr = female_eer_fp/(female_eer_tn+female_eer_fp)

male_eer_tnr = male_eer_tn/(male_eer_tn+male_eer_fp)
male_eer_tpr = male_eer_tp/(male_eer_tp+male_eer_fn)
male_eer_fnr = male_eer_fn/(male_eer_fn+male_eer_tp)
male_eer_fpr = male_eer_fp/(male_eer_tn+male_eer_fp)

print(female_eer_fnr/male_eer_fnr)
print(female_eer_fpr/male_eer_fpr)
print(male_eer_tnr/female_eer_tnr)
print(male_eer_tpr/female_eer_tpr)
print((1-female_eer_acc)/(1-male_eer_acc))
print('female fnr:',female_eer_fnr,'male fnr:',male_eer_fnr)
print('female fpr:',female_eer_fpr,'male fpr:',male_eer_fpr)
print('female tnr:',male_eer_tnr,'male tnr:',female_eer_tnr)
print('female tpr:',male_eer_tpr,'male tpr:',female_eer_tpr)
print('female mis class:',(1-female_eer_acc),'male mis class:',(1-male_eer_acc))

In [56]:
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)
gender_val
val_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
val_bias_df['label']=y_val.cpu().numpy()
val_bias_df['prediction']=val_cos.cpu().numpy()
val_bias_df['rounded']=eer_pred.cpu().numpy()
val_bias_df['protected']=gender_val
val_bias_df

,label,prediction,protected,rounded
0,1.0,0.757461,0,1
1,0.0,0.225750,0,0
2,0.0,0.395677,0,0
3,1.0,0.783808,0,1
4,1.0,0.766834,0,1
...,...,...,...,...
7582,0.0,0.315627,0,0
7583,0.0,0.232081,0,0
7584,1.0,0.739232,1,1
7585,0.0,0.368275,1,0


In [57]:
unpriveleged_df = val_bias_df[val_bias_df['protected']==1]
unpriveleged_df

,label,prediction,protected,rounded
5,0.0,0.313320,1,0
6,0.0,0.308538,1,0
11,0.0,0.318686,1,0
14,0.0,0.283172,1,0
15,1.0,0.570144,1,1
...,...,...,...,...
7575,1.0,0.615757,1,1
7578,1.0,0.629097,1,1
7579,0.0,0.301008,1,0
7584,1.0,0.739232,1,1


In [58]:
priveleged_df = val_bias_df[val_bias_df['protected']==0]
priveleged_df

,label,prediction,protected,rounded
0,1.0,0.757461,0,1
1,0.0,0.225750,0,0
2,0.0,0.395677,0,0
3,1.0,0.783808,0,1
4,1.0,0.766834,0,1
...,...,...,...,...
7580,1.0,0.690326,0,1
7581,1.0,0.842168,0,1
7582,0.0,0.315627,0,0
7583,0.0,0.232081,0,0


In [59]:
unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['rounded'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['rounded'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum(eer_pred == y_val).item()/y_val.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.953
Missclassificaition Rate Ratio: 1.391
FNR Ratio: 1.186 FPR Ratio: 1.674
AOE: 0.015


In [60]:
def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.astype(np.float32)
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset

In [61]:
aif_data = to_dataframe(y_val, eer_pred, gender_val)

In [62]:
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}],seed=random_state)

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print(np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOE:',round(AOE(eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp,eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp),3))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [74]:
cost_constraint = 'weighted'
privileged_groups=[{'y_prot': 0.}],
unprivileged_groups=[{'y_prot': 1.}]
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint,seed=random_state)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

unpriv_ids = (cpp_prot==1)
unpriv_pred = cpp_pred[unpriv_ids]
unpriv_label = cpp_label[unpriv_ids]

priv_ids = (cpp_prot==0)
priv_pred = cpp_pred[priv_ids]
priv_label = cpp_label[priv_ids]

unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp, unpriv_acc = confusion_mat(unpriv_pred, unpriv_label)
unpriv_fpr = unpriv_fp/(unpriv_fp+unpriv_tn)
unpriv_fnr = unpriv_fn/(unpriv_fn+unpriv_tp)

priv_tn, priv_fp, priv_fn, priv_tp, priv_acc = confusion_mat(priv_pred, priv_label)
priv_fpr = priv_fp/(priv_fp+priv_tn)
priv_fnr = priv_fn/(priv_fn+priv_tp)

print('Missclassificaition Rate Ratio:',round((1-unpriv_acc)/(1- priv_acc),3))
print('FPR Ratio:',round(unpriv_fpr/priv_fpr,3))
print('FNR Ratio:',round(unpriv_fnr/priv_fnr,3))
print('Overall Accuracy:',round(np.sum(cpp_pred == cpp_label)/len(cpp_label),3))
print('AOE:',round(AOE(unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp,priv_tn, priv_fp, priv_fn, priv_tp),3))

Missclassificaition Rate Ratio: 0.966
FPR Ratio: 1.787
FNR Ratio: 0.657
Overall Accuracy: 0.944
AOE: 0.026


In [70]:
ROC = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.3, high_class_thresh=0.8,
                                         num_class_thresh=100, num_ROC_margin=100,
                                         metric_name="Average odds difference",
                                         metric_ub=0.01, metric_lb=-0.01)

ROC = ROC.fit(aif_data, aif_data)

ROC_pred = ROC.predict(aif_data).labels.reshape(-1)
ROC_label = aif_data.labels.reshape(-1)
ROC_prot = aif_data.protected_attributes.reshape(-1)

ROC_unpriv_ids = (ROC_prot==1)
ROC_unpriv_pred = ROC_pred[ROC_unpriv_ids]
ROC_unpriv_label = ROC_label[ROC_unpriv_ids]

ROC_priv_ids = (ROC_prot==0)
ROC_priv_pred = ROC_pred[ROC_priv_ids]
ROC_priv_label = ROC_label[ROC_priv_ids]

ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp, ROC_unpriv_acc = confusion_mat(ROC_unpriv_pred, ROC_unpriv_label)
ROC_unpriv_fpr = ROC_unpriv_fp/(ROC_unpriv_fp+ROC_unpriv_tn)
ROC_unpriv_fnr = ROC_unpriv_fn/(ROC_unpriv_fn+ROC_unpriv_tp)

ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp, ROC_priv_acc = confusion_mat(ROC_priv_pred, ROC_priv_label)
ROC_priv_fpr = ROC_priv_fp/(ROC_priv_fp+ROC_priv_tn)
ROC_priv_fnr = ROC_priv_fn/(ROC_priv_fn+ROC_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-ROC_unpriv_acc)/(1- ROC_priv_acc),3))
print('FPR Ratio:',round(ROC_unpriv_fpr/ROC_priv_fpr,3))
print('FNR Ratio:',round(ROC_unpriv_fnr/ROC_priv_fnr,3))
print(np.sum(ROC_pred == ROC_label)/len(ROC_pred))
print('AOE:',round(AOE(ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp,ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp),3))


Missclassificaition Rate Ratio: 1.391
FPR Ratio: 1.674
FNR Ratio: 1.186
0.9529458283906682
AOE: 0.015
